<a href="https://colab.research.google.com/github/Ahmed-Mustafa-2005/Computer-Forensics/blob/main/Malware_Detection_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import pathlib
import copy
from torch.utils.tensorboard import SummaryWriter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
!unrar x "/content/drive/My Drive/malware_dataset/train.tar" -C "/content/drive/My Drive/data/"

Here we download necessary library to split folders


In [ ]:
!pip install split-folders

This will kill created data folder (only use if you want restart the whole model creation process)

In [ ]:
import shutil

shutil.rmtree('/content/drive/My Drive/data/')

In [ ]:
import splitfolders
splitfolders.ratio('/content/drive/My Drive/data/train', output="output", seed=1337, ratio=(.8, 0.2))

In [ ]:
# how many samples per batch to load
data_dir= pathlib.Path(os.path.normpath('/content/output/'))
train_path = data_dir/"train"
val_path = data_dir/"val"

In [ ]:
malware_train = len(os.listdir(train_path/"malware"))
bengin_train = len(os.listdir(train_path/"benign"))
malware_valid = len(os.listdir(val_path/"malware"))
bengin_valid = len(os.listdir(val_path/"benign"))
print("Malware Train Images: {}".format(malware_train))
print("Bengin Train Images: {}".format(bengin_train))
print("Malware Validaten Images: {}".format(malware_valid))
print("Bengin Validation Images: {}".format(bengin_valid))

In [ ]:
transform = transforms.Compose([transforms.Resize((256,256)),
                               transforms.ToTensor(),
                               transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

In [ ]:
#loading the folders
train = datasets.ImageFolder(train_path, transform = transform)
val = datasets.ImageFolder(val_path, transform=transform)

In [ ]:
#loading the dataset
batch_size = 128
train_set = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle = True)
val_set = torch.utils.data.DataLoader(val, batch_size = batch_size, shuffle=True)

In [ ]:
classes = os.listdir(train_path)
print(classes)

In [ ]:
def imshow(image, title = None):
    image = image.numpy().transpose((1,2,0))
    mean = np.array([0.485,0.456,0.406])
    std = np.array([0.229,0.224,0.225])
    image = image*std + mean
    image = np.clip(image, 0, 1)
    plt.imshow(image)
    if title is not None:
        print(title)
    plt.pause(0.001)

In [ ]:
#getting a batch of training data
images, labels = next(iter(train_set))
out = torchvision.utils.make_grid(images)
imshow(out,title=[classes[x] for x in labels])

In [ ]:
imshow(images[1],classes[labels[0]])

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # convolutions
        self.conv1 = nn.Conv2d(3,16,3,padding=1)
        self.conv2 = nn.Conv2d(16,32,3,padding=1)
        self.conv3 = nn.Conv2d(32,64,3,padding=1)
        self.conv4 = nn.Conv2d(64,128,3,padding=1)
        self.conv5 = nn.Conv2d(128,64,3,padding=1)
        # Max-pool
        self.pool = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(256*4*4,1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 2)
        #self.fc4 = nn.Linear(1024, 512)
        #self.fc5 = nn.Linear(512, 2)
        # Dropout module with 0.5 drop probability
        self.dropout = nn.Dropout(p = 0.5, inplace=True)

    def forward(self, x):
        # make sure input tensor is flattened
        x = self.pool(F.relu(self.conv1(x)))
        print(x.shape)
        x = self.pool(F.relu(self.conv2(x)))
        print(x.shape)
        x = self.pool(F.relu(self.conv3(x)))
        print(x.shape)
        x = self.pool(F.relu(self.conv4(x)))
        print(x.shape)
        x = self.pool(F.relu(self.conv5(x)))
        print(x.shape)
        x = x.view(-1,256*4*4)

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        #x = self.dropout(F.relu(self.fc3(x)))
        #x = self.dropout(F.relu(self.fc4(x)))
        # output with softmax
        x = F.log_softmax(self.fc3(x),dim=1)

        return x

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

model.to(device)

In [ ]:
writer = SummaryWriter()

In [ ]:
writer = SummaryWriter()
dataiter = iter(train_set)
images, labels = dataiter.next()
writer.add_graph(model, images.to(device))
writer.close()

In [ ]:
!pip install torchsummary
from torchsummary import summary
model = model.to(device)
summary(model, input_size= (3,256,256))

In [ ]:
from itertools import product
parameters = dict(
    dropout = [0.1, 0.3, 0.5],
    lr = [0.01, 0.001],
    batch_size = [32,64,128],
    shuffle = [True, False]
)

param_val = [v for v in parameters.values()]
print(param_val)

for dropout, lr,batch_size, shuffle in product(*param_val):
    print(dropout, lr, batch_size, shuffle)

In [ ]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [ ]:
for hyperpam_test_id, (dropout, lr,batch_size, shuffle) in enumerate(product(*param_val)):
    print("Hyperparameter Test ID:", hyperpam_test_id + 1)
    model = Net().to(device)
    train_set = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = shuffle)
    optimizer = optim.Adam(model.parameters(), lr= lr)
    criterion = torch.nn.CrossEntropyLoss()
    comment = f'dropout = {dropout} batch_size = {batch_size} lr = {lr} shuffle = {shuffle}'
    writer = SummaryWriter(comment=comment)
    for epoch in range(10):
        total_loss = 0
        total_correct = 0
        for images, labels in train_set:
            images, labels = images.to(device), labels.to(device)
            preds = model(images)

            loss = criterion(preds, labels)
            total_loss+= loss.item()
            total_correct+= get_num_correct(preds, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        writer.add_scalar("Loss", total_loss, epoch)
        writer.add_scalar("Correct", total_correct, epoch)
        writer.add_scalar("Accuracy", total_correct/ len(train_set), epoch)

        print("dropout:",dropout, "batch_size:",batch_size, "lr:",lr,"shuffle:",shuffle)
        print("epoch:", epoch, "total_correct:", total_correct, "loss:",total_loss)
    print("___________________________________________________________________")

    writer.add_hparams(
            {"dropout" : dropout, "lr": lr, "batch_size": batch_size, "shuffle":shuffle},
            {
                "accuracy": total_correct/ len(train_set),
                "loss": total_loss,
            },
        )

writer.close()

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir ./runs/

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
scheduler = lr_scheduler.StepLR(optimizer, step_size = 7, gamma=0.1)

In [ ]:
history={'train_loss':[],'valid_loss':[],'train_acc':[],'valid_acc':[]}
num_epochs=10


for epoch in range(num_epochs):
    train_loss,train_correct=0.0,0
    model.train()
    for images, labels in train_set:

        images,labels = images.to(device),labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        scores, predictions = torch.max(output.data, 1)
        train_correct += (predictions == labels).sum().item()

    valid_loss, val_correct = 0.0, 0
    model.eval()
    for images, labels in val_set:

        images,labels = images.to(device),labels.to(device)
        output = model(images)
        loss=criterion(output,labels)
        valid_loss+=loss.item()*images.size(0)
        scores, predictions = torch.max(output.data,1)
        val_correct+=(predictions == labels).sum().item()


    train_loss = train_loss / len(train_set.sampler)
    train_acc = train_correct / len(train_set.sampler)*100
    valid_loss = valid_loss / len(val_set.sampler)
    valid_acc = val_correct / len(val_set.sampler) * 100

    print("Epoch:{}/{} \t average training loss:{:.4f} average validation loss:{:.4f} \t average training acc.:{:.2f} %  average validation acc.:{:.2f} %".format(epoch + 1, num_epochs,
                                                                                             train_loss,
                                                                                             valid_loss,
                                                                                             train_acc,
                                                                                            valid_acc))
    history['train_loss'].append(train_loss)
    history['valid_loss'].append(valid_loss)
    history['train_acc'].append(train_acc)
    history['valid_acc'].append(valid_acc)


In [ ]:
import shutil
shutil.rmtree("./runs/")

Here you can define a confusion matrix and later reflect within tensorboard

In [ ]:
 nb_classes = 2

 confusion_matrix = torch.zeros(nb_classes, nb_classes)
 with torch.no_grad():
     for i, data in enumerate(val_set, 0):
         images, labels = data
         images = images.to(device)
         labels = labels.to(device)
         outputs = model(images)
         _, preds = torch.max(outputs, 1)
         for t, p in zip(labels.view(-1), preds.view(-1)):
                 confusion_matrix[t.long(), p.long()] += 1

cm = confusion_matrix.numpy()
cm = cm / cm.sum(axis=1)

import seaborn as sns

classes = ("bengin", "malware")

fig = plt.figure(figsize=(10, 7))
heatmap = sns.heatmap( cm, annot=True, cmap="gray" ,
                       xticklabels = classes, yticklabels=classes)

plt.ylabel("True Label")
plt.xlabel("Predicted Label")



In [ ]:
writer = SummaryWriter()
writer.add_figure("Confusion Matrix", fig)
writer.close()

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir ./runs/